In [1]:
import cv2
import numpy as np

def detect_eyes(image):
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    eyes = eye_cascade.detectMultiScale(image, scaleFactor=1.3, minNeighbors=15)
    return eyes

In [9]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [17]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    frame_filter = cv2.medianBlur(frame, 9)
    # frame_filter = cv2.GaussianBlur(frame, (5, 5), 0)
    # frame_filter = cv2.bilateralFilter(frame, d=12, sigmaColor=15, sigmaSpace=15)
    
    # intensity_brightness = cv2.convertScaleAbs(frame, alpha=1.5, beta=20)
    
    cv2.imshow("Imagem Original", frame)
    cv2.imshow("Imagem Filtrada", frame_filter)    
    # cv2.imshow("Imagem Corrigida", intensity_brightness) 

    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [19]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    cv2.imshow("Imagem Original", frame)
    cv2.imshow("Imagem em Escala de Cinza", gray)    

    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [27]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    eyes = detect_eyes(gray)
    for (ex, ey, ew, eh) in eyes:
        roi_gray = gray[ey:ey+eh, ex:ex+ew]

    
    cv2.imshow("Imagem Original", frame) 
    cv2.imshow("Imagem olho", roi_gray) 

    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [35]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.convertScaleAbs(gray, alpha=1.5, beta=20)

    eyes = detect_eyes(gray)
    for (ex, ey, ew, eh) in eyes:
        roi_gray = gray[ey:ey+eh, ex:ex+ew]
        roi_gray_filtered = cv2.bilateralFilter(roi_gray, d=9, sigmaColor=75, sigmaSpace=75)

        threshold = cv2.adaptiveThreshold(roi_gray_filtered, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 5)
        threshold = cv2.medianBlur(threshold, 9)

        kernel = np.ones((2, 1), np.uint8)
        threshold = cv2.erode(threshold, kernel, iterations=1)

        threshold = cv2.bitwise_not(threshold)
        contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        cor = (255, 0, 0)
        for contour in contours:
            cv2.drawContours(frame, [contour + (ex, ey)], 0, cor, -1)
            

    # cv2.imshow("Imagem olho", roi_gray) 
    # cv2.imshow("Imagem olho filtrado", roi_gray_filtered)
    cv2.imshow("Imagem segmentado", threshold)  
    cv2.imshow("Contornos", frame) 

    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [60]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.convertScaleAbs(gray, alpha=1.5, beta=20)

    eyes = detect_eyes(gray)
    for (ex, ey, ew, eh) in eyes:
        roi_gray = gray[ey:ey+eh, ex:ex+ew]
        roi_gray_filtered = cv2.bilateralFilter(roi_gray, d=9, sigmaColor=75, sigmaSpace=75)
        
        threshold = cv2.adaptiveThreshold(roi_gray_filtered, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 5)
        threshold = cv2.medianBlur(threshold, 9)

        kernel = np.ones((2, 1), np.uint8)
        threshold = cv2.erode(threshold, kernel, iterations=1)

        threshold = cv2.bitwise_not(threshold)
        contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        cor = (255, 0, 0)
        for contour in contours:
            area = cv2.contourArea(contour)
            perimeter = cv2.arcLength(contour, True)
            try:
                ratio = area / perimeter
            except ZeroDivisionError:
                ratio = 0

            if ratio > 1.5:
                cv2.drawContours(frame, [contour + (ex, ey)], 0, cor, -1)
            

    # cv2.imshow("Imagem olho", roi_gray) 
    # cv2.imshow("Imagem olho filtrado", roi_gray_filtered)
    cv2.imshow("Imagem segmentado", threshold)  
    cv2.imshow("Contornos", frame) 

    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [74]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.convertScaleAbs(gray, alpha=1.5, beta=20)

    eyes = detect_eyes(gray)
    for (ex, ey, ew, eh) in eyes:
        largest_contour = None
        max_ratio = 0.0

        roi_gray = gray[ey:ey+eh, ex:ex+ew]
        roi_gray_filtered = cv2.bilateralFilter(roi_gray, d=9, sigmaColor=75, sigmaSpace=75)
        
        threshold = cv2.adaptiveThreshold(roi_gray_filtered, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 5)
        threshold = cv2.medianBlur(threshold, 9)

        kernel = np.ones((2, 1), np.uint8)
        threshold = cv2.erode(threshold, kernel, iterations=1)

        threshold = cv2.bitwise_not(threshold)
        contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        cor = (255, 0, 0)
        for contour in contours:
            area = cv2.contourArea(contour)
            perimeter = cv2.arcLength(contour, True)
            try:
                ratio = area / perimeter
            except ZeroDivisionError:
                ratio = 0

            if ratio > max_ratio and area > (10+ex*0.001) and ratio > 0.2:
                max_ratio = ratio
                largest_contour = contour 

        cor = (0, 0, 255)
        if largest_contour is not None:
            cv2.drawContours(frame, [largest_contour + (ex, ey)], 0, cor, -1)
            

    # cv2.imshow("Imagem olho", roi_gray) 
    # cv2.imshow("Imagem olho filtrado", roi_gray_filtered)
    # cv2.imshow("Imagem segmentado", threshold)  
    # cv2.imshow("Contornos", frame) 
    
    frame_filter = cv2.bilateralFilter(frame, d=12, sigmaColor=15, sigmaSpace=15)

    # Exibe as duas imagens lado a lado
    cv2.imshow("Pupila com Filtro", frame_filter)
    cv2.imshow("Imagem Original", frame)

    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()